# Import

In [81]:
import pandas as pd
from pathlib import  Path
import  matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import  numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.metrics import r2_score

# Path and Read File

In [39]:
DATA_DIR=Path("../data/houseprice.csv")
data=pd.read_csv(DATA_DIR)
data

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33
...,...,...,...,...,...,...,...,...
3474,86,2,True,True,True,Southern Janatabad,3.500000e+09,116666.67
3475,83,2,True,True,True,Niavaran,6.800000e+09,226666.67
3476,75,2,False,False,False,Parand,3.650000e+08,12166.67
3477,105,2,True,True,True,Dorous,5.600000e+09,186666.67


# Data Cleaning

In [40]:
def extract_neighborhood(address):
    parts = address.split(',')
    if len(parts) > 0:
        return parts[0].strip()
    else:
        return 0

In [79]:
data["Area"]=pd.to_numeric(data["Area"],errors='coerce')
data["Area"]=data["Area"].dropna()
data["Area"]=data["Area"].astype(float)
df_area=data["Area"]
Q1=df_area.quantile(0.25)
Q3=df_area.quantile(0.75)
IQR=Q3-Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data_clean=data[(data["Area"] >= lower_bound) & (data["Area"] <= upper_bound)]


data_clean=data_clean.dropna()
data_clean = data_clean.drop("Price",axis=1)

# data_clean["Address"]=data_clean["Address"].apply(lambda x:extract_neighborhood(x))
# data_clean=pd.get_dummies(data_clean,columns=["Address"])


addresses=data_clean["Address"]
neighborhoods = [extract_neighborhood(addr) for addr in addresses]
le = LabelEncoder()
neighborhoods_encoded = le.fit_transform(neighborhoods)

data_clean["Address"]=neighborhoods_encoded
data_clean["Parking"]=data_clean["Parking"].astype(int)
data_clean["Warehouse"]=data_clean["Warehouse"].astype(int)
data_clean["Elevator"]=data_clean["Elevator"].astype(int)



address_price_mean = data_clean.groupby('Address')['Price(USD)'].mean().sort_values(ascending=True)
address_ranking = {address: rank for rank, address in enumerate(address_price_mean.index, 1)}
data_clean['Address_Rank'] = data_clean['Address'].map(address_ranking)
data_clean.drop('Address', axis=1, inplace=True)


data_clean

,Area,Room,Parking,Warehouse,Elevator,Price(USD),Address_Rank
0,63.0,1,1,1,1,61666.67,102
1,60.0,1,1,1,1,61666.67,102
2,79.0,2,1,1,1,18333.33,17
3,95.0,2,1,1,1,30083.33,28
4,123.0,2,1,1,1,233333.33,182
...,...,...,...,...,...,...,...
3474,86.0,2,1,1,1,116666.67,110
3475,83.0,2,1,1,1,226666.67,184
3476,75.0,2,0,0,0,12166.67,7
3477,105.0,2,1,1,1,186666.67,178


# Train and Test

In [80]:
data_y=data_clean["Price(USD)"]
data_x=data_clean.drop("Price(USD)",axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=101)

# Model fit

In [102]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[0]=X_train_scaled[0]*0.35  #area
X_train_scaled[1]=X_train_scaled[1]*0.025   #room
X_train_scaled[2]=X_train_scaled[2]*0.05   #parking
X_train_scaled[3]=X_train_scaled[3]*0.025   #warehouse
X_train_scaled[4]=X_train_scaled[4]*0.05   #elevator
X_train_scaled[5]=X_train_scaled[5]*0.50   #addrees rank

y_train_log = np.log(y_train)

model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train_scaled, y_train_log)
y_pred_log = model.predict(X_test_scaled)
y_pred = np.exp(y_pred_log)


In [103]:
print(f"R²: {r2_score(y_test, y_pred)}")

R²: 0.8317763225838568


In [ ]:
print("salam")